In [115]:
import wandb
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import argparse
import pandas as pd
import torch
import matplotlib as mpl
mpl.rcParams['axes.grid'] = True
from scipy.optimize import curve_fit

plt.rcParams.update({
    "font.size": 12,          # controls default text sizes
    "axes.titlesize": 14,     # fontsize of the axes title
    "axes.labelsize": 13,     # fontsize of the x and y labels
    "xtick.labelsize": 11,    # fontsize of the tick labels
    "ytick.labelsize": 11,
    "legend.fontsize": 11,    # legend text size
    "figure.titlesize": 14,   # overall figure title
})

from scipy.stats import norm
from collections import defaultdict
import matplotlib.lines as mlines
import matplotlib.patches as mpatches


In [116]:
PROJECT = "alexa_-ista/llm-baselines"   # <- edit
MAX_RUNS = 1000                          # how many recent runs to grab
DATASET_NAME = "c4"

wandb.login(key="01b6cad29dd97d55aa6132d283623075386d6d73")  


api   = wandb.Api()
runs  = api.runs(PROJECT,
                 filters={ "config.dataset": DATASET_NAME,
                          "config.w_quant": "OutliersQuestQuantizer",
                           "state": "finished" },
                 )[:MAX_RUNS]



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /nfs/scistore19/alistgrp/avolkova/.netrc
wandb: Currently logged in as: alexa_ (alexa_-ista) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [117]:
model_sizes = np.array([30482560, 49556736, 102777856, 203188480, 443068288, 822151168])

model_sizes_dict = {30: 30482560, 50: 49556736, 100: 102777856, 200: 203188480, 400: 443068288, 800: 822151168}

n_layer_to_num_params = {6: 30482560, 7: 49556736, 8: 102777856, 10: 203188480, 12: 443068288, 14: 822151168}

In [118]:
bit_levels = [run.config["w_quant_kwargs"]["bits"] for run in runs]
bit_levels = list(set(bit_levels))
outlier_ratio_levels = [run.config["w_quant_kwargs"]["outlier_ratio"] for run in runs]
outlier_ratio_levels = list(set(outlier_ratio_levels))

bit_levels, outlier_ratio_levels

([1, 2, 3, 4, 5, 6, 8], [0, 0.01, 0.05, 0.02])

In [119]:
OPTIMAL_GAUSSIAN_SCALES = {
    1: 0.7978845587140913,
    1.585: 1.2240089519030855,
    2: 1.4935346200015913,
    3: 2.051068354131873,
    4: 2.513930578568423,
    5: 2.9160938834961225,
    6: 3.276597282593217,
    7: 3.6010497188221655,
    8: 3.884938678807525,
}

In [120]:
def mse_tails(bits, outlier_ratio):
    
    x_had = torch.randn(1024*1024)

    n_levels = 2**bits
    std = torch.sqrt(torch.mean(x_had**2, dim=-1, keepdim=True))
    scale = OPTIMAL_GAUSSIAN_SCALES[bits] * std + 1e-8
    step = 2 * scale / (n_levels - 1)

    outlier_thresh = norm.ppf(1 - outlier_ratio/2)

    tails_mask = (x_had.abs() > outlier_thresh * std).float()

            # x_clip = torch.clamp(x_had * (1 - tails_mask), -scale, scale)

    xq = torch.round(x_had / step + 1 / 2) * step - step / 2
    trust = OPTIMAL_GAUSSIAN_SCALES[bits] / (n_levels - 1)

    mask = (torch.abs(xq - x_had) <= std * trust).float()

    x_out = xq * (1 - tails_mask) + x_had*tails_mask
    mse = ((x_out - x_had) ** 2).mean()
    return mse


In [121]:
mse_tails(4, 0.0), mse_tails(4, 0.05)

(tensor(0.0094), tensor(0.0088))

In [122]:
colors = plt.cm.plasma(np.linspace(0, 0.9, len(bit_levels)))

In [123]:
A=10.533504165111173
alpha=0.11128093
B=440.00059095898183
beta=0.32784117
E=1.3205800739096627

def loss_law(X, eff_p):
    N = X
    D = N * 100
    return A / (N * eff_p)**alpha + E + B/(D)**beta

def tanh_law(X, L, F, C):
    MSE = X
    L = 1.0
    denom = L * np.tanh(F * (- np.log2(MSE + 1e-8) / 2))**C
    denom = np.clip(denom, 1e-8, None)   # never let it go below 1e-8

    return denom


In [124]:
rho = defaultdict(lambda: np.zeros(len(bit_levels)))

for id_outl, outliers_ratio in enumerate(outlier_ratio_levels):
    runs_subset = [run for run in runs if run.config["w_quant_kwargs"]["outlier_ratio"] == outliers_ratio]

    for id_b, bits in enumerate(bit_levels):
        model_sizes = np.empty(0)
        losses = np.empty(0)
        mse_arr = np.empty(0)
        runs_by_size = [run for run in runs_subset if run.config["w_quant_kwargs"]["bits"] == bits]
        model_sizes = np.concatenate((model_sizes,[n_layer_to_num_params[run.config["n_layer"]] for run in runs_by_size]))

        losses = np.concatenate((losses, [run.summary["final-val/loss"] for run in runs_by_size]))

        mse_val = np.full((len(runs_by_size)),  mse_tails(bits, outliers_ratio))
        mse_arr = np.append(mse_arr, mse_val)

        # print(len(losses), len(model_sizes), len(mse_arr))
        if len(losses) >= 1:
            param, err = curve_fit(loss_law, model_sizes, losses)
            # print(param, np.sqrt(np.diag(err)))
            rho[outliers_ratio][id_b] = param[0]
rho

/tmp/ipykernel_2072231/61474551.py:10: RuntimeWarning: invalid value encountered in power
  return A / (N * eff_p)**alpha + E + B/(D)**beta
/tmp/ipykernel_2072231/1200676611.py:20: OptimizeWarning: Covariance of the parameters could not be estimated
  param, err = curve_fit(loss_law, model_sizes, losses)


defaultdict(<function __main__.<lambda>()>,
            {0: array([0.29914376, 0.44978206, 0.64564114, 0.81443579, 0.91979965,
                    0.97783752, 0.9947156 ]),
             0.01: array([0.29793921, 0.4470677 , 0.66386465, 0.83616307, 0.95491718,
                    1.00223576, 1.03422655]),
             0.05: array([0.29803032, 0.4478117 , 0.66496382, 0.83185267, 0.95435313,
                    1.00472564, 1.03195923]),
             0.02: array([0.29123944, 0.43722464, 0.64401   , 0.        , 0.        ,
                    0.        , 0.        ])})

In [125]:
outlier_ratio_levels.sort()

In [126]:
mpl.rcParams['axes.grid.which'] = 'major'

plt.figure(figsize=(6, 4))

x = np.zeros(len(bit_levels))
y = np.zeros((len(bit_levels), 2))

outlier_ratio_to_plot = [0.0, 0.05]

colors = plt.cm.plasma(np.linspace(0.2, 0.9, len(outlier_ratio_levels)))
markers = ['o', 's', 'x', 'P', 'X', 'H']

for id_outl, outliers_ratio in enumerate(outlier_ratio_levels):
    if outliers_ratio not in outlier_ratio_to_plot:
        continue
    for id_b, bits in enumerate(bit_levels):
        if rho[outliers_ratio][id_b] > 0.0:
            plt.scatter(mse_tails(bits, outliers_ratio), rho[outliers_ratio][id_b], 
                        color=colors[id_outl], marker=markers[id_outl], s=100,
                        linewidths=0.8, edgecolors='black',)

    # for id_b, bits in enumerate(bit_levels):
    #     if y[id_b][1] != float('inf'):
    #         plt.errorbar(x[id_b], y[id_b][0], yerr=y[id_b][1],)
    #     plt.scatter(x[id_b], y[id_b][0],
    #                 color=colors[id_b], label=f"{bits}", zorder=2,
    #                 edgecolor='black', s=80, linewidths=0.6)

handles = []
handles.append(mpatches.Patch(color='none', label='Outliers:'))

for id_outl, outliers_ratio in enumerate(outlier_ratio_levels):
    if outliers_ratio not in outlier_ratio_to_plot:
        continue
    handles.append(
        mlines.Line2D([], [], color=colors[id_outl], marker=markers[id_outl], linestyle='None',
                        markersize=9, markeredgewidth=0.8,markeredgecolor='black', label=f"{outliers_ratio*100}%")
    )
leg = plt.legend(handles=handles,  ncol=1)


x = np.empty(len(bit_levels))
y = np.empty(len(bit_levels))
for id_b, b in enumerate(bit_levels):
    x[id_b] = mse_tails(b, 0.0)
    y[id_b] = rho[0.0][id_b]

param, err = curve_fit(tanh_law, x, y)
print(param, np.sqrt(np.diag(err)))

x_support = np.logspace(-4.2, -0.5, 100)
plt.plot(x_support, tanh_law(x_support, *param), color='gray', label="tanh fit", linestyle='--', linewidth=1.3, zorder=0)

plt.xscale("log")
plt.xlabel("MSE")
plt.ylabel(r"$\rho(R)$")
# plt.ylim(top=1.01)
# plt.legend(title="Outlier ratio:",)
plt.xticks([1e-4, 1e-3, 1e-2, 1e-1,])
plt.grid(zorder=0)
plt.tight_layout()
plt.savefig("../plots/outliers.pdf", bbox_inches='tight')
plt.show()


/tmp/ipykernel_2072231/61474551.py:15: RuntimeWarning: invalid value encountered in power
  denom = L * np.tanh(F * (- np.log2(MSE + 1e-8) / 2))**C
/tmp/ipykernel_2072231/829462679.py:48: OptimizeWarning: Covariance of the parameters could not be estimated
  param, err = curve_fit(tanh_law, x, y)


[1.         0.39523283 1.50422022] [inf inf inf]
